In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3) #input -? OUtput? RF
        self.conv1_bn=nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.conv2_bn=nn.BatchNorm2d(16)
        self.dropout = nn.Dropout(p=0.01)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 16, 3)
        self.conv3_bn=nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(16, 64, 3)
        self.conv4_bn=nn.BatchNorm2d(64)
        self.conv5 = nn.Conv2d(64, 10, 1)
        self.avgpool = nn.AvgPool2d(3)

    def forward(self, x):
        x=self.conv1(x)
        x = F.relu(self.conv1_bn(x))
        x = self.conv2(x)
        x = F.relu(self.conv2_bn(x))
        x = self.dropout(x)
        x = self.pool1(x)
        x = self.conv3(x)
        x = F.relu(self.conv3_bn(x))
        x = self.pool2(x)
        x = self.conv4(x)
        x = F.relu(self.conv4_bn(x))
        x = self.conv5(x)
        x = self.avgpool(x)
        x = x.view(-1, 10)

        return F.log_softmax(x)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
            Conv2d-3           [-1, 16, 24, 24]           1,168
       BatchNorm2d-4           [-1, 16, 24, 24]              32
           Dropout-5           [-1, 16, 24, 24]               0
         MaxPool2d-6           [-1, 16, 12, 12]               0
            Conv2d-7           [-1, 16, 10, 10]           2,320
       BatchNorm2d-8           [-1, 16, 10, 10]              32
         MaxPool2d-9             [-1, 16, 5, 5]               0
           Conv2d-10             [-1, 64, 3, 3]           9,280
      BatchNorm2d-11             [-1, 64, 3, 3]             128
           Conv2d-12             [-1, 10, 3, 3]             650
    

<ipython-input-14-00888ecd4993>:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:


torch.manual_seed(1)
batch_size = 16

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    print(f'epoch"{epoch}')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

epoch"1


  0%|          | 0/3750 [00:00<?, ?it/s]<ipython-input-14-00888ecd4993>:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.04056926816701889 batch_id=3749: 100%|██████████| 3750/3750 [00:46<00:00, 81.33it/s]



Test set: Average loss: 0.0400, Accuracy: 9864/10000 (98.6400%)

epoch"2


loss=0.033669613301754 batch_id=3749: 100%|██████████| 3750/3750 [00:42<00:00, 88.59it/s]



Test set: Average loss: 0.0318, Accuracy: 9906/10000 (99.0600%)

epoch"3


...


Test set: Average loss: 0.0201, Accuracy: 9943/10000 (99.4300%)

epoch"19


loss=4.390216781757772e-05 batch_id=3749: 100%|██████████| 3750/3750 [00:40<00:00, 92.27it/s]



Test set: Average loss: 0.0235, Accuracy: 9931/10000 (99.3100%)

epoch"20


loss=0.004821481183171272 batch_id=3749: 100%|██████████| 3750/3750 [00:40<00:00, 91.89it/s]



Test set: Average loss: 0.0205, Accuracy: 9941/10000 (99.4100%)

